# '조문'에서 수행주체 출력

In [2]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import pickle

In [3]:
tqdm.pandas()

## 1. 수행주체를 list로 저장한 subject_list.pkl 로드

In [4]:
with open("subject_list.pkl","rb") as f:
    subject_list = pickle.load(f)

In [5]:
subject_list[:50]

['대도시',
 '2023 순천만국제정원박람회조직위원회',
 '5·18민주화운동 진상규명조사위원회',
 '계약담당자',
 '대검찰청',
 '대한민국법인',
 '가석방심사위원회',
 '가정보호사건조사관',
 '가정위탁지원센터',
 '가축방역기관장',
 '가축병성감정실시기관',
 '가축전염병피해보상협의회',
 '기관',
 '본부',
 '부대',
 '학교',
 '장관',
 '간사',
 '갈등관리심의위원회',
 '감독기관',
 '감독청',
 '감사',
 '감사관',
 '감사원',
 '감사위원회',
 '감사활동을 수행하는 사람',
 '감염병관리위원회',
 '감염병병원체 확인기관',
 '감정평가법인',
 '감항인증심의위원회',
 '개발센터',
 '개발원',
 '갱생보호회지소장',
 '거래소',
 '거주자',
 '건강보험공단',
 '건강보험분쟁조정위원회',
 '건강보험심사평가원',
 '건설기계사업자설립협회',
 '건설사업자',
 '건설엔지니어링사업자',
 '건설청',
 '건축위원회',
 '건축주',
 '건축허가관청',
 '검사',
 '검사공무원',
 '검사위원회',
 '검역기관',
 '검역본부']

In [4]:
df = pd.read_csv('BaseTable4.csv')

/var/folders/sy/5dw5r1ys5fdb3h0gbq8x0g6m0000gn/T/ipykernel_67754/4038033726.py:1: DtypeWarning: Columns (4,5,18,19,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('BaseTable4.csv')


## 2. 'BaseTable4'의 '조문'에서 수행주체 뽑아내기

In [17]:
# "조문"이 결측치인 행 제거
df['조문'] = df['조문'].apply(lambda x: '' if pd.isna(x) else str(x))

In [18]:
# 수행주체 앞에 붙을 수 있는 단어
prefixes = ['각 목의', '각급', '각군', '각부', '각종', '관계', '관할', '소관', '소속', '해당']

# 수행주체 뒤에 붙을 수 있는 단어
suffixes = ['의 장', '장', '의 위원장', '총장', '장관', '위원장', '이 아닌 자', '이사장', '의장',
            '시장', '회장', '의 의장', '의 본부장', '의 원장', '회장', '등의 장']

# "수행주체 앞에 붙을 수 있는 단어 + 수행주체 + 수행주체 뒤에 붙을 수 있는 단어" 미리 붙여서 후보 만들어놓기
compiled_patterns = []
for subject in subject_list:
    pattern_list = [re.escape(prefix) + r"\s*" + re.escape(subject) + r"\s*" for prefix in prefixes]
    pattern_list += [re.escape(subject) + r"\s*" + re.escape(suffix) + r"\s*" for suffix in suffixes]
    pattern_list.append(re.escape(subject))
    compiled_patterns += [re.compile(pattern) for pattern in pattern_list]

def find_subjects_in_text(text):
    subjects_found = set()

    for pattern in compiled_patterns:
        match = pattern.search(text)
        if match:
            found_subject = ' '.join(match.group().split())  # 공백 정규화
            subjects_found.add(found_subject)

    # 더 긴 주체가 포함하는 짧은 주체 제거
    for subject1 in list(subjects_found):
        for subject2 in list(subjects_found):
            if subject1 != subject2 and subject1 in subject2:
                subjects_found.discard(subject1)

    return ', '.join(subjects_found)

# 'new_수행주체' 열 생성
df['new_수행주체'] = df['조문'].progress_apply(find_subjects_in_text)

100%|██████████| 861621/861621 [1:26:37<00:00, 165.79it/s]


In [23]:
# 수행주체 추가하여 basetable 생성
df.to_csv('BaseTable4_new_수행주체추가.csv')

## 3. BaseTable4_new_수행주체추가.csv 살펴보기

In [3]:
df_add_subject = pd.read_csv('BaseTable4_new_수행주체추가.csv')

/var/folders/sy/5dw5r1ys5fdb3h0gbq8x0g6m0000gn/T/ipykernel_84304/2884985943.py:1: DtypeWarning: Columns (5,6,19,20,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_add_subject = pd.read_csv('BaseTable4_new_수행주체추가.csv')


In [7]:
df_add_subject[(df_add_subject['new_수행주체'].notnull()) & (df_add_subject['수행주체'].notnull())][['조문', '수행주체', 'new_수행주체']]

,조문,수행주체,new_수행주체
35,① 개인정보 보호에 관한 사무를 독립적으로 수행하기 위하여 국무총리 소속으로 개인정...,국무총리,"국무총리, 보호위원회"
92,① 보호위원회는 다음 각 호의 사항을 심의ㆍ의결한다.,보호위원회,보호위원회
137,① 중앙행정기관의 장은 소관 법령의 제정 또는 개정을 통하여 개인정보 처리를 수반하...,중앙행정기관의 장,"중앙행정기관의 장, 보호위원회"
141,① 보호위원회는 개인정보의 보호와 정보주체의 권익 보장을 위하여 3년마다 개인정보 ...,보호위원회,"중앙행정기관의 장, 관계 중앙행정기관, 보호위원회"
152,① 중앙행정기관의 장은 기본계획에 따라 매년 개인정보 보호를 위한 시행계획을 작성하...,중앙행정기관의 장,"중앙행정기관의 장, 보호위원회"
...,...,...,...
860705,⑤제1항부터 제4항까지의 규정에 따른 과태료는 대통령령으로 정하는 바에 따라 환경부...,환경부장관,"군수, 시장, 구청장, 환경부장관, 대통령, 시ㆍ도지사"
860754,① 환경부장관은 제11조에 따라 상수원보호구역을 지정하거나 변경하려는 경우에는 주민...,환경부장관,환경부장관
861016,④위탁심의위원회의 위원은 다음 각 호의 자 중에서 일반수도사업자가 위촉한다. 이 경...,일반수도사업자(지방자치단체 등),"일반수도사업자, 심의위원회"
861208,①환경부장관은 협회의 업무를 감독한다.,환경부장관,"협회, 환경부장관"


In [9]:
# '수행주체'열과 조문에서 출력한 'new_수행주체'열 비교
pd.crosstab(df_add_subject['수행주체'].notnull(), df_add_subject['new_수행주체'].notnull())

new_수행주체,False,True
수행주체,,
False,410651,390904
True,796,59270


[결과 분석] <br/>
- 원 데이터에서는 '수행주체' 없는데, 위의 코드를 통해 'new_수행주체' 출력된 행: 390904개 <br/>
- 원 데이터에서는 '수행주체' 있는데, 위의 코드를 통해 'new_수행주체' 출력되지 않은 행: 796개 <br/>
⇨ 총 391,700개